# Week 1 HW1_Regression_TaihwanOh

오태환 1주차 과제

# Q1

이 경우 Threshold를 0.3으로 낮추어 acceptance rate가 매우 높다. 즉 0.3 이상이면 True로 본다는 뜻이다. 따라서 H_0를 True로 더 많이 판정하게 된다. 따라서 H_0이 not True임에도 True라고 판정하는 경우(False Positive)가 많아지므로 Type 2 Error가 더 높다고 할 수 있을 것 같다.

# Q2-1

$$Accuracy = \frac{TP + TN}{TP + FN + FP + TN}$$

$$ Precision = \frac{TP}{TP+FP}$$

$$ Recall = \frac{TP}{FN + TP}$$

# Q2-2

ESC는 작년 파이널 프로젝트로 파산 기업 예측 프로젝트를 진행했다. 파산 기업을 예측할 때 중요한 것은 파산하는 기업을 잡아내는 것이다. 파산하는 기업이 은행에 더 많은 손해를 끼치기 때문이다. 그렇기 때문에 실제로 파산하지 않더라도 파산으로 예측하는 비율을 늘리는 것을 감수하고도 최대한 파산할 회사를 많이 예측하는 것이 중요하다. 즉 Recall을 높이는 것이 중요하다. 이 예시에서 threshold를 늘린다는 것의 의미는 파산으로 예측하는 기준 파산 확률을 높인다는 뜻이므로 파산으로 예측하는 비율을 더 낮춘다는 것을 의미한다. 그렇기 때문에 비합리적이라고 생각한다.

# Q3

In [8]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [11]:
# Hyper Parameters  
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

데이터셋 불러오기

In [12]:
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  # 이럴로 train 데이터 불러오기
                            transform = transforms.ToTensor(), # 이걸로 텐서
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  # 이걸로 test데이터 다운로드
                           transform = transforms.ToTensor()) 
  
# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True) #랜덤으로 뽑아서 배치 만들기
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False) 

In [13]:
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) # 1 layer perceptron
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 


In [14]:
model = LogisticRegression(input_size, num_classes) 

In [15]:
criterion = nn.CrossEntropyLoss() # multi class 문제이므로 CE loss 사용
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) 
# optimizer는 SGD(stochastic gradient descent)를 쓴다

In [17]:
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28)) 
        labels = Variable(labels) 
  
        # Forward + Backward + Optimize 
        optimizer.zero_grad() 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train_dataset) // batch_size, loss.data.item())) 

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.1094
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.0178
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 1.9505
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.8744
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.8288
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.6997
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.6013
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.5616
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.4907
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5579
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.4216
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4204
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.3585
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.2957
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.2327
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.3307
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.2406
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.1330
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2528
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.2107
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [18]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
  
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 

Accuracy of the model on the 10000 test images:  83 %


..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


# Q4-1

optim.SGD를 사용하지 않으려면 이런 방식을 쓰면 된다.

$$ \frac{1}{m}\sum_{i=1} (H(x^i)-y^i)^2$$

이 cost의 gradient를 learning rate만큼 곱해 빼주면서 업데이트 하자. 코드는 아래와 같다

hypothesis = x_train * W 로 initial hypothesis를 만든 후

cost = torch.mean((hypothesis - y_train) ** 2) 로 cost를 만든다.

gradient = torch.sum((W * x_train - y_train) * x_train) 으로 gradient 계산 후

W -= lr * gradient로 업데이트하는 것을 반복하자

# Q4-2

In [20]:
criterion = nn.CrossEntropyLoss() # multi class 문제이므로 CE loss 사용
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) 
# optimizer는 Adam을 쓴다

In [21]:
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28)) 
        labels = Variable(labels) 
  
        # Forward + Backward + Optimize 
        optimizer.zero_grad() 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train_dataset) // batch_size, loss.data.item())) 

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 0.6399
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 0.4551
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 0.4013
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 0.3227
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 0.3218
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 0.2855
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 0.2966
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 0.3384
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 0.2063
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 0.2004
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 0.2429
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 0.2189
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 0.3457
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 0.2568
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 0.2491
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 0.3972
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 0.2486
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 0.3762
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 0.2636
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 0.1986
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [22]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
  
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 

Accuracy of the model on the 10000 test images:  92 %


Accuracy가 92%로 더 좋아졌다!